In [1]:
import cv2
import numpy as np
import pandas as pd
import itertools

In [2]:
##### tarefas para fazer
## Algoritmos de detecção
# -- harris - OK
# -- sift - OK
# -- star/brisk OK
# -- entender parâmetros harris, sift, star e brisk (pendente)
# -- criar experimento para 25, 50, 100, 500, 1000 features (abandonado)
########################
## Algoritmos de descrição
# -- restringir área de checagem de pontos 
# -- criar algoritmo para contar falsos positivos - OK
## Imagem de teste
# -- preciso escolher qual o terceiro par de imagens para testar -OK

# Experimento com detectores

# Códigos detectores

In [3]:
def detectHarrisKeypoints(image, threshold=0.01, blockSize=2, ksize=3, k=0.04):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
    gray_image_f32 = np.float32(gray_image)

    # Applying the function 
    dst = cv2.cornerHarris(gray_image_f32, blockSize, ksize, k) 
  
    # dilate to mark the corners 
    dst = cv2.dilate(dst, None)
    
    ret, dst = cv2.threshold(dst,threshold*dst.max(),255,0)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(gray_image_f32,np.float32(centroids),(5,5),(-1,-1),criteria)

    # # extract keypoints
    # points = np.argwhere(dst > threshold * dst.max())
    
    keypoints = [cv2.KeyPoint(float(x[0]), float(x[1]), 13) for x in corners]

    # draw keypoints
    # image[dst > threshold * dst.max()] = [0, 255, 0]
    kp_image = cv2.drawKeypoints(image, keypoints, None, color=(255, 0, 0), flags=0)

    return keypoints, kp_image


In [4]:
def detectSIFTKeypoints(image, nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6, enable_precise_upscale=False):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Applying the function 
    sift = cv2.SIFT_create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma, enable_precise_upscale) 
    kp, des = sift.detectAndCompute(gray_image, None) 
    
    # Applying the function 
    kp_image = cv2.drawKeypoints(image, kp, None, color=(0, 255, 0), flags=0) 

    return kp, kp_image 

In [5]:
def detectStarKeypoints(image, max_size = 41, response_threshold = 30, line_threshold_projected = 10,
                        line_threshold_binarized = 8, suppress_nonmax_size = 5):
    # Reading the image and converting the image to B/W 
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) 
  
    # Applying the function 
    star = cv2.xfeatures2d.StarDetector_create(maxSize= max_size, 
                                        responseThreshold = response_threshold,
                                        lineThresholdProjected = line_threshold_projected,
                                        lineThresholdBinarized = line_threshold_binarized,
                                        suppressNonmaxSize = suppress_nonmax_size)
    kp = star.detect(gray_image, None)    

    # Applying the function 
    kp_image = cv2.drawKeypoints(image, kp, None, color=(0, 0, 255), flags=0) 

    return kp, kp_image 

In [6]:
# image = cv2.imread('../data/imgs/dsc07631.jpg')
# kp, kp_image = detectHarrisKeypoints(image, threshold=0.01, blockSize=2, ksize=3, k=0.02)
# print(len(kp))
# cv2.imshow('Star', kp_image) 
# cv2.waitKey() 

## Código comparando pontos

In [7]:
import numpy as np
from sklearn.metrics import pairwise_distances_argmin_min
from joblib import Parallel, delayed

def computeDistacesKeypoints(pts1, pts2, threshold=1):
    array_pts1 = np.asarray(pts1)
    array_pts2 = np.asarray(pts2)

    if array_pts1.shape[0] > 0 and array_pts2.shape[0] > 0:
        dists = pairwise_distances_argmin_min(array_pts1, array_pts2)  
        matches = [pts2[pt] for pt, dist in zip(dists[0], dists[1]) if dist <= threshold]
    else:
        matches = []
        
    return matches

In [8]:
def extract_and_save_features_csv(image, params, algol, path):
    if algol == 'star':
        func_var = detectStarKeypoints
    elif algol == 'harris':
        func_var = detectHarrisKeypoints
    else:
        func_var = detectSIFTKeypoints
    
    cols_name = [algol + '_' + param for param in list(params.keys())[1:]]
    cols_result = [algol + '_' + 'qtd_keypoints', algol + '_' + 'keypoints']
    df = pd.DataFrame(columns=cols_name+cols_result)
    
    keys = list(params)
    for values in itertools.product(*map(params.get, keys)):
        kps, kp_image = func_var(image, **dict(zip(keys, values)))
        kps_ = [(kp.pt[0], kp.pt[1]) for kp in kps] #kp.angle, kp.response, kp.octave, kp.class_id) for kp in kps]

        df.loc[len(df)] = list(values[1:]) + [len(kps_), kps_]

    df.to_csv(path, index=False)
    return df


def convert_to_kp(string_points):
    string_converted = list(eval(string_points))
    # kps = [cv2.KeyPoint(x[0], x[1], 13) for x in string_converted]
    # for p in string_converted:
    #     kp = cv2.KeyPoint(x=float(p[0]), y=float(p[1]), size=float(p[2]), angle=float(p[3]),
    #                 response=float(p[4]), octave=int(p[5]), class_id=int(p[6]))
    #     kps.append(kp)
    
    return string_converted


def save_image_with_matches(image, kp1, kp2, kp_matches, path, name_img):
    kp_image = image.copy()
    path_img = f'{path}{name_img}.jpg' 

    kp1_ = [kp for kp in np.asarray(kp1) if kp not in np.asarray(kp_matches)]
    kp2_ = [kp for kp in np.asarray(kp2) if kp not in np.asarray(kp_matches)]

    kp1_ = [cv2.KeyPoint(x[0], x[1], 13) for x in kp1_]
    kp2_ = [cv2.KeyPoint(x[0], x[1], 13) for x in kp2_]
    kp_matches = [cv2.KeyPoint(x[0], x[1], 13) for x in kp_matches]

    kp_image = cv2.drawKeypoints(kp_image, kp1_, None, color=(0, 0, 255), flags=0)
    kp_image = cv2.drawKeypoints(kp_image, kp2_, None, color=(255, 0, 0), flags=0)
    kp_image = cv2.drawKeypoints(kp_image, kp_matches, None, color=(0, 255, 0), flags=0)
    
    cv2.imwrite(path_img, kp_image)

def comparing_keypoints_parallel(image, df, df1, df2, idx1, kp1_name, kp2_name, cols1, cols2, extra_cols, path):
    kp1_col = f'{kp1_name}_keypoints'
    kp2_col = f'{kp2_name}_keypoints'

    kp1 = df1.loc[idx1][kp1_col]
    df = pd.DataFrame(columns=cols1+cols2+extra_cols)
    for idx2 in df2.index:
        kp2 = df2.loc[idx2][kp2_col]
        matches = computeDistacesKeypoints(kp1, kp2, threshold=2)
        
        # save_image_with_matches(image, kp1, kp2, matches, path, len(df))
        
        df.loc[len(df)] = df1.loc[idx1][cols1].tolist() + df2.loc[idx2][cols2].tolist() + [len(matches), matches]
    return df

def compare_and_save_match_points(image, df1, df2, kp1_name, kp2_name, path):
    cols1 = list(df1.columns[:-1])
    cols2 = list(df2.columns[:-1])
    extra_cols = ['qtd_matches', 'match_keypoints']

    df1_ = df1.copy()
    df2_ = df2.copy()

    df = pd.DataFrame(columns=cols1+cols2+extra_cols)
    dfs = []
    dfs = Parallel(n_jobs=-1)(delayed(comparing_keypoints_parallel)(image, df, df1_, df2_, idx1, kp1_name, kp2_name, cols1, cols2, extra_cols, path) for idx1 in df1_.index)
    df = pd.concat(dfs, ignore_index=True)
    
    path_csv = f'{path}matches_{kp1_name}_{kp2_name}.csv' 
    df.to_csv(path_csv, index=False)

    return df

## Salvando combinações de parâmetros dos algoritmos

In [9]:
params_harris = {
    # 'image': [image],
    'threshold' : [0.005, 0.05, 0.1, 0.2],
    'blockSize' : [2, 4, 6, 8],
    'ksize' : [3, 5, 7, 9],
    'k' : [0.02, 0.04, 0.08, 0.16]
}

params_sift = {
    # 'image': [image],
    # 'nfeatures': [50, 100, 500, 1000],
    'nOctaveLayers' : [1, 3, 5, 7],
    'contrastThreshold' : [0.02, 0.04, 0.08, 0.1],
    'edgeThreshold' : [5, 10, 20, 40],
    'sigma' : [0.8, 1.6, 3.2, 6.4]
}

params_star = {
    # 'image': [image],
    'max_size': [11, 21, 41, 81],
    'response_threshold': [5, 10, 20, 30],
    'line_threshold_projected': [5, 10, 20, 30],
    'line_threshold_binarized': [4, 8, 16, 32],
    'suppress_nonmax_size': [2, 3, 5, 7]
}



In [10]:
train_img_1 = cv2.imread('../data/imgs_teste/IMG_20231217_075101.jpg')
train_img_2 = cv2.imread('../data/imgs/dsc07631.jpg')
train_img_3 = cv2.imread('../data/imgs/dsc02595.jpg')

train_img_1_ = cv2.resize(train_img_1, (480,640))
train_img_2_ = cv2.resize(train_img_2, (480,640))
train_img_3_ = cv2.resize(train_img_3, (480,640))

In [11]:
name_imgs = ['IMG_20231217_075101', 'dsc07631', 'dsc02595']
train_imgs = [train_img_1_, train_img_2_, train_img_3_]

# for image, name_img in zip(train_imgs, name_imgs):
#     df_harris = extract_and_save_features_csv(image, params_harris, 'harris', f'../results/{name_img}/harris_keypoints.csv')
#     df_sift = extract_and_save_features_csv(image, params_sift, 'sift', f'../results/{name_img}/sift_keypoints.csv')
#     df_star = extract_and_save_features_csv(image, params_star, 'star', f'../results/{name_img}/star_keypoints.csv')

In [1324]:
name_imgs = [
    # 'IMG_20231217_075101',
      'dsc07631', 'dsc02595']
train_imgs = [
    # train_img_1_,
      train_img_2_, train_img_3_]

for name_img, train_img in zip(name_imgs, train_imgs):
    df_harris = pd.read_csv(f'../results/{name_img}/harris_keypoints.csv')
    df_sift = pd.read_csv(f'../results/{name_img}/sift_keypoints.csv')
    df_star = pd.read_csv(f'../results/{name_img}/star_keypoints.csv')

    kps1 = []
    for idx in df_harris.index.tolist():
        kps1.append(convert_to_kp(df_harris['harris_keypoints'].loc[idx]))

    kps2 = []
    for idx in df_sift.index.tolist():
        kps2.append(convert_to_kp(df_sift['sift_keypoints'].loc[idx]))

    kps3 = []
    for idx in df_star.index.tolist():
        kps3.append(convert_to_kp(df_star['star_keypoints'].loc[idx]))

    df_harris['harris_keypoints'] = kps1
    df_sift['sift_keypoints'] = kps2
    df_star['star_keypoints'] = kps3

    df_hs = compare_and_save_match_points(train_img, df_harris, df_sift, 'harris', 'sift', f'../results/{name_img}/kp_harris_sift/')
    df_hst = compare_and_save_match_points(train_img, df_harris, df_star, 'harris', 'star', f'../results/{name_img}/kp_harris_star/')
    df_ss = compare_and_save_match_points(train_img, df_sift, df_star, 'sift', 'star', f'../results/{name_img}/kp_sift_star/')
    

In [1287]:
kps1 = []
for idx in df_harris.index.tolist():
    kps1.append(convert_to_kp(df_harris['harris_keypoints'].loc[idx]))
    

In [1288]:
kps2 = []
for idx in df_sift.index.tolist():
    kps2.append(convert_to_kp(df_sift['sift_keypoints'].loc[idx]))

In [1289]:
kps3 = []
for idx in df_star.index.tolist():
    kps3.append(convert_to_kp(df_star['star_keypoints'].loc[idx]))

In [1290]:
df_harris['harris_keypoints'] = kps1
df_sift['sift_keypoints'] = kps2
df_star['star_keypoints'] = kps3

In [ ]:
df_hs = compare_and_save_match_points(train_img_2, df_harris, df_sift, 'harris', 'sift', '../results/dsc07631/kp_harris_sift/')

In [59]:
df_hst = compare_and_save_match_points(df_harris, df_star, 'harris_keypoints', 'star_keypoints', '../data/results/dsc07631/harris_star_matches.csv')

In [60]:
df_ss = compare_and_save_match_points(df_sift, df_star, 'sift_keypoints', 'star_keypoints', '../data/results/dsc07631/sift_star_matches.csv')

## Segundo Experimento

In [389]:
df_harris.head()

,harris_threshold,harris_blockSize,harris_ksize,harris_k,harris_qtd_keypoints,harris_keypoints
0,0.01,2,3,0.02,10493,"[(71.0, 0.0), (72.0, 0.0), (73.0, 0.0), (74.0,..."
1,0.01,2,3,0.04,9288,"[(71.0, 0.0), (72.0, 0.0), (73.0, 0.0), (74.0,..."
2,0.01,2,3,0.16,5947,"[(98.0, 2.0), (99.0, 2.0), (100.0, 2.0), (98.0..."
3,0.01,2,5,0.02,8844,"[(71.0, 0.0), (72.0, 0.0), (73.0, 0.0), (74.0,..."
4,0.01,2,5,0.04,7534,"[(71.0, 0.0), (72.0, 0.0), (73.0, 0.0), (74.0,..."


In [ ]:
matches_hs = computeDistacesKeypoints(h_kp, s_kp, threshold=2)
matches_ho = computeDistacesKeypoints(h_kp, o_kp, threshold=2)
matches_so = computeDistacesKeypoints(s_kp, o_kp, threshold=2)

## Primeiro experimento
- Rodando com os parâmetros default dos algoritmos

In [271]:
image = cv2.imread('../data/imgs/dsc07631.jpg')
h_kp, harris_image = detectHarrisKeypoints(image)
s_kp, sift_image = detectSIFTKeypoints(image)
o_kp, orb_image = detectStarKeypoints(image)


### threshold para matches de detectores 1

In [272]:
matches_hs = computeDistacesKeypoints(h_kp, s_kp, threshold=1)
matches_ho = computeDistacesKeypoints(h_kp, o_kp, threshold=1)
matches_so = computeDistacesKeypoints(s_kp, o_kp, threshold=1)

In [258]:
print('----- Resultados -----')
print(f'QTD Harris Keypoints:   {len(h_kp)}')
print(f'QTD SIFT Keypoints:   {len(s_kp)}')
print(f'QTD ORB Keypoints:   {len(o_kp)}')
print('----- Matches -----')
print(f'Matches Keypoints Harris/SIFT:   {len(matches_hs)}')
print(f'Matches Keypoints Harris/ORB:   {len(matches_ho)}')
print(f'Matches Keypoints SIFT/ORB:   {len(matches_so)}')

----- Resultados -----
QTD Harris Keypoints:   9288
QTD SIFT Keypoints:   1518
QTD ORB Keypoints:   500
----- Matches -----
Matches Keypoints Harris/SIFT:   645
Matches Keypoints Harris/ORB:   666
Matches Keypoints SIFT/ORB:   80


In [259]:
kp_matches_hs = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_hs]
kp_matches_ho = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_ho]
kp_matches_so = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_so]

In [260]:
kp1_image = cv2.drawKeypoints(image, kp_matches_hs, None, color=(0, 0, 255), flags=0)
kp2_image = cv2.drawKeypoints(image, kp_matches_ho, None, color=(0, 255, 0), flags=0)
kp3_image = cv2.drawKeypoints(image, kp_matches_so, None, color=(255, 0, 0), flags=0)

In [262]:
# cv2.imshow('ORB', kp_matches_hs)

### threshold para matches de detectores 2

In [263]:
matches_hs = computeDistacesKeypoints(h_kp, s_kp, threshold=2)
matches_ho = computeDistacesKeypoints(h_kp, o_kp, threshold=2)
matches_so = computeDistacesKeypoints(s_kp, o_kp, threshold=2)

In [264]:
print('----- Resultados -----')
print(f'QTD Harris Keypoints:   {len(h_kp)}')
print(f'QTD SIFT Keypoints:   {len(s_kp)}')
print(f'QTD ORB Keypoints:   {len(o_kp)}')
print('----- Matches -----')
print(f'Matches Keypoints Harris/SIFT:   {len(matches_hs)}')
print(f'Matches Keypoints Harris/ORB:   {len(matches_ho)}')
print(f'Matches Keypoints SIFT/ORB:   {len(matches_so)}')

----- Resultados -----
QTD Harris Keypoints:   9288
QTD SIFT Keypoints:   1518
QTD ORB Keypoints:   500
----- Matches -----
Matches Keypoints Harris/SIFT:   2342
Matches Keypoints Harris/ORB:   1430
Matches Keypoints SIFT/ORB:   125


In [265]:
kp_matches_hs = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_hs]
kp_matches_ho = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_ho]
kp_matches_so = [cv2.KeyPoint(x[0], x[1], 13) for x in matches_so]

In [266]:
kp1_image = cv2.drawKeypoints(image, kp_matches_hs, None, color=(0, 0, 255), flags=0)
kp2_image = cv2.drawKeypoints(image, kp_matches_ho, None, color=(0, 255, 0), flags=0)
kp3_image = cv2.drawKeypoints(image, kp_matches_so, None, color=(255, 0, 0), flags=0)

In [268]:
# cv2.imshow('Harris/SIFT', kp1_image)
# cv2.waitKey() 

# Comparando Descritores

In [1161]:
import copy

def calculate_slope(pt1, pt2):
    slope = (pt2[1] - pt1[1]) / (pt2[0] - pt1[0]) if (pt2[0] - pt1[0]) != 0 else -1
    return slope


def verify_slope(match, trainKeypoints, queryKeypoints, slope1, slope2):
    slope = calculate_slope(trainKeypoints[match.trainIdx].pt, queryKeypoints[match.queryIdx].pt) 
    if slope <= slope2 and slope >= slope1:
        return True
    else:
        return False


def verify_dist(match, trainKeypoints, queryKeypoints, dist1, dist2):
    point1 = np.asarray(trainKeypoints[match.trainIdx].pt)
    point2 = np.asarray(queryKeypoints[match.queryIdx].pt)

    dist = np.linalg.norm(point1 - point2)

    if dist <= dist2 and dist >= dist1:
        return True
    else:
        return False


def calculate_slope_list(matches, matchesMask, trainKeypoints, queryKeypoints):
    slope_list = [calculate_slope(trainKeypoints[matches[i][0].trainIdx].pt, queryKeypoints[matches[i][0].queryIdx].pt) for i, m in enumerate(matchesMask)]
    slope_list_mask = [calculate_slope(trainKeypoints[matches[i][0].trainIdx].pt, queryKeypoints[matches[i][0].queryIdx].pt) for i, m in enumerate(matchesMask) if m[0] > 0]

    return slope_list, slope_list_mask


def calculate_dist_list(matches, matchesMask, trainKeypoints, queryKeypoints):
    kps_train = [(trainKeypoints[matches[i][0].trainIdx].pt[0], trainKeypoints[matches[i][0].trainIdx].pt[1]) for i, m in enumerate(matchesMask)]
    kps_query = [(queryKeypoints[matches[i][0].queryIdx].pt[0], queryKeypoints[matches[i][0].queryIdx].pt[1]) for i, m in enumerate(matchesMask)]

    kps_train_ = [(trainKeypoints[matches[i][0].trainIdx].pt[0], trainKeypoints[matches[i][0].trainIdx].pt[1]) for i, m in enumerate(matchesMask) if m[0] > 0]
    kps_query_ = [(queryKeypoints[matches[i][0].queryIdx].pt[0], queryKeypoints[matches[i][0].queryIdx].pt[1]) for i, m in enumerate(matchesMask) if m[0] > 0]
    
    array_pts_train = np.asarray(kps_train)
    array_pts_query = np.asarray(kps_query)

    array_pts_train_ = np.asarray(kps_train_)
    array_pts_query_ = np.asarray(kps_query_)

    dists = [np.linalg.norm(point1 - point2) for point1, point2 in zip(array_pts_train, array_pts_query)]
    dists_mask = [np.linalg.norm(point1 - point2) for point1, point2 in zip(array_pts_train_, array_pts_query_)]

    return dists, dists_mask


def filter_matches_by_percentile(matches, matchesMask, trainKeypoints, queryKeypoints, q1, q2):
    slope_list, slope_list_ = calculate_slope_list(matches, matchesMask, trainKeypoints, queryKeypoints)
    dists, dists_ = calculate_dist_list(matches, matchesMask, trainKeypoints, queryKeypoints)

    s1_, s2_ = np.percentile(slope_list_, [q1 ,q2])
    d1_, d2_ = np.percentile(dists_, [q1 ,q2])
    
    matchesMask_ = [ [1,0] if m[0] > 0 and verify_slope(matches[i][0], trainKeypoints, queryKeypoints, s1_, s2_) else [0,0] for i, m in enumerate(matchesMask)]
    matchesMask_ = [ [1,0] if m[0] > 0 and verify_dist(matches[i][0], trainKeypoints, queryKeypoints, d1_, d2_) else [0,0] for i, m in enumerate(matchesMask_)]

    qtd_matches = len(matches)
    qtd_matches_dlowe = len([m[0] for m in matchesMask if m[0] > 0 ])
    qtd_matches_filter = len([m[0] for m in matchesMask_ if m[0] > 0 ])

    qtd_fp_dlowe = qtd_matches - qtd_matches_dlowe
    qtd_fp_filter = qtd_matches_dlowe - qtd_matches_filter

    var_dist = np.var([d for m, d in zip(matchesMask, dists) if m[0] > 0 ])
    var_slope = np.var([sl for m, sl in zip(matchesMask, slope_list) if m[0] > 0 ])

    return matchesMask_, qtd_matches, qtd_matches_dlowe, qtd_matches_filter, qtd_fp_dlowe, qtd_fp_filter, var_dist, var_slope


def filter_matches_by_variance(matches, trainKeypoints, queryKeypoints, var_dist, var_slope):
    matchesMask_ = [[0,0] for i in range(len(matches))]
    var_dist_ = -1
    var_slope_ = -1
    ratio_test_ = -1
    
    for ratio_test in np.arange(0.1, 1., 0.01):
        matchesMask = [[0,0] for i in range(len(matches))]

        for i,(m,n) in enumerate(matches):
            if m.distance < ratio_test*n.distance:
                matchesMask[i]=[1,0]

        slope_list, slope_list_ = calculate_slope_list(matches, matchesMask, trainKeypoints, queryKeypoints)
        dists, dists_ = calculate_dist_list(matches, matchesMask, trainKeypoints, queryKeypoints)

        var_d = np.var(dists_)
        var_s = np.var(slope_list_)

        if var_d <= var_dist and var_s <= var_slope:
            ratio_test_ = round(copy.copy(ratio_test), 2)
            var_dist_ = var_d
            var_slope_ = var_s
            matchesMask_ = [m for m in matchesMask]
        
    qtd_matches = len(matches)
    qtd_matches_dlowe = len([m[0] for m in matchesMask_ if m[0] > 0 ])

    qtd_fp_dlowe = qtd_matches - qtd_matches_dlowe

    return matchesMask_, ratio_test_, qtd_matches, qtd_matches_dlowe, qtd_fp_dlowe, var_dist_, var_slope_


def draw_matches(train_img, query_img, trainKeypoints, queryKeypoints, matches, matchesMask):
    kps_train = [[trainKeypoints[matches[i][0].trainIdx].pt[0], trainKeypoints[matches[i][0].trainIdx].pt[1]] for i, m in enumerate(matchesMask)]
    kps_query = [[queryKeypoints[matches[i][0].queryIdx].pt[0], queryKeypoints[matches[i][0].queryIdx].pt[1]] for i, m in enumerate(matchesMask)]

    # draw the tracks
    mask = np.zeros_like(train_img, 'uint8') 

    line = (0,255,0)
    point_true = (255,0,0)
    point_false = (0,0,255)
    frame = query_img.copy()
    
    for i, (m, query, train) in enumerate(zip(matchesMask, kps_query, kps_train)):
        a, b = query[0], query[1]
        c, d = train[0], train[1]
        
        a, b = int(a), int(b)
        c, d = int(c), int(d)
        
        if m[0] > 0:
            mask = cv2.line(mask, (a, b), (c, d), line, 2) 
            frame = cv2.circle(frame, (a, b), 4, point_true, -1)
        else:
            frame = cv2.circle(frame, (a, b), 4, point_false, -1)
    
    img = cv2.add(frame, mask)

    return img


In [1162]:
def matchDetectedKeypoints(train_img, query_img, var_dist, var_slope, detector='harris', descriptor='orb', dect_kargs={}, des_kargs={}):
    query_img_bw = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY) 
    train_img_bw = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)

    if descriptor == 'brief':
        des = cv2.xfeatures2d.BriefDescriptorExtractor_create(**des_kargs)
        norm_type = cv2.NORM_HAMMING
    elif descriptor == 'brisk':
        des = cv2.BRISK_create(**des_kargs)
        norm_type = cv2.NORM_HAMMING
    else:
        des = cv2.SIFT.create(**des_kargs)
        norm_type = cv2.NORM_L2
    
    if detector == 'star':
        func_var = detectStarKeypoints
    elif detector == 'harris':
        func_var = detectHarrisKeypoints
    else:
        func_var = detectSIFTKeypoints

    queryKeypoints, _ = func_var(query_img, **dect_kargs)
    trainKeypoints, _ = func_var(train_img, **dect_kargs)
    
    _,queryDescriptors = des.compute(query_img_bw, queryKeypoints) 
    _,trainDescriptors = des.compute(train_img_bw, trainKeypoints)


    matcher = cv2.BFMatcher(normType=norm_type, crossCheck=False) 
    matches = matcher.knnMatch(queryDescriptors,trainDescriptors, k=2) 

    # Need to draw only good matches, so create a mask
    # matchesMask = [[0,0] for i in range(len(matches))]

    # for i,(m,n) in enumerate(matches):
    #     if m.distance < ratio_test*n.distance:
    #         matchesMask[i]=[1,0]

    # m = filter_matches_by_percentile(matches, matchesMask, trainKeypoints, queryKeypoints, filter_slope[0], filter_slope[1])
    
    m = filter_matches_by_variance(matches, trainKeypoints, queryKeypoints, var_dist, var_slope)
    matchesMask_, ratio_test_, qtd_matches, qtd_matches_dlowe, qtd_fp_dlowe, var_dist_, var_slope_ = m 

    draw_params = dict(matchColor = (0,255,0),
                    singlePointColor = (255,0,0),
                    matchesMask = matchesMask_,
                    flags = cv2.DrawMatchesFlags_DEFAULT)
    
    # final_img = cv2.drawMatchesKnn(query_img,queryKeypoints,train_img,trainKeypoints,matches,None,**draw_params)
    qtd_kp_train = len(trainKeypoints)
    qtd_kp_query = len(queryKeypoints)

    final_img = draw_matches(train_img, query_img, trainKeypoints, queryKeypoints, matches, matchesMask_)

    final_img = cv2.resize(final_img, (480,640))

    return final_img, qtd_kp_train, qtd_kp_query, ratio_test_, var_dist_, var_slope_, qtd_matches, qtd_matches_dlowe, qtd_fp_dlowe 

    

In [1195]:
def detect_and_save_result_matches_csv(train_img, query_img, var_dist, var_slope,
                                       detector, descriptor, params_detector, path):
    
    cols_name = [param for param in list(params_detector.keys())]
    cols_result = ['qtd_kp_train', 'qtd_kp_query', 'ratio_test', 'var_dist', 'var_slope',
                    'qtd_matches', 'qtd_matches_dlowe', 'qtd_fp_dlowe']

    df = pd.DataFrame(columns=cols_name+cols_result)
    
    keys = list(params_detector)
    for i, values in enumerate(itertools.product(*map(params_detector.get, keys))):
        try:
            result = matchDetectedKeypoints(train_img, query_img, var_dist, var_slope,
                                        detector, descriptor, dict(zip(keys, values)))
        except:
            result = [-1] + [-1 for i in range(len(cols_result))]

        df.loc[len(df)] = list(values) + list(result[1:])
        
        path_img = path + f'matches_{detector}_{descriptor}/'+ str(i) + '.jpg'
        cv2.imwrite(path_img, result[0])
    
    path_csv = path + f'matches_{detector}_{descriptor}/'+ f'matches_{detector}_{descriptor}.csv'
    df.to_csv(path_csv)
    
    return df

In [1196]:
params_harris = {
    'threshold' : [0.005, 0.05, 0.1, 0.2],
    'blockSize' : [2, 4, 6, 8],
    'ksize' : [3, 5, 7, 9],
    'k' : [0.02, 0.04, 0.08, 0.16]
}

params_sift = {
    # 'nfeatures': [50, 100, 500, 1000],
    'nOctaveLayers' : [1, 3, 5, 7],
    'contrastThreshold' : [0.02, 0.04, 0.08, 0.1],
    'edgeThreshold' : [5, 10, 20, 40],
    'sigma' : [0.8, 1.6, 3.2, 6.4]
}

params_star = {
    'max_size': [11, 21, 41, 81],
    'response_threshold': [5, 10, 20, 30],
    'line_threshold_projected': [5, 10, 20, 30],
    'line_threshold_binarized': [4, 8, 16, 32],
    'suppress_nonmax_size': [2, 3, 5, 7]
}

var_img_pacoca = (1, 0.001) # IMG_20231217_075107.jpg
var_img1 = (4, 0.02) # dsc07632.jpg
var_img2 = (3, 0.05) # dsc02596.jpg


In [1197]:
query_img_1 = cv2.imread('../data/imgs_teste/IMG_20231217_075107.jpg') 
train_img_1 = cv2.imread('../data/imgs_teste/IMG_20231217_075101.jpg')

query_img_2 = cv2.imread('../data/imgs/dsc07632.jpg') 
train_img_2 = cv2.imread('../data/imgs/dsc07631.jpg')

query_img_3 = cv2.imread('../data/imgs/dsc02596.jpg') 
train_img_3 = cv2.imread('../data/imgs/dsc02595.jpg')

query_img_1_ = cv2.resize(query_img_1, (480,640))
train_img_1_ = cv2.resize(train_img_1, (480,640))

query_img_2_ = cv2.resize(query_img_2, (480,640))
train_img_2_ = cv2.resize(train_img_2, (480,640))

query_img_3_ = cv2.resize(query_img_3, (480,640))
train_img_3_ = cv2.resize(train_img_3, (480,640))

In [1198]:
name_imgs = ['IMG_20231217_075101', 'dsc07631', 'dsc02595']
train_imgs = [train_img_1_, train_img_2_, train_img_3_]
query_imgs = [query_img_1_, query_img_2_, query_img_3_]
variances = [var_img_pacoca, var_img1, var_img2]
params_detectors = [params_harris,  params_star, params_sift]
detectors = ['harris', 'star', 'sift']
descriptors = ['sift', 'brisk']

for name_img, var, train_img, query_img in zip(name_imgs, variances,
                                                        train_imgs, query_imgs):
     path = f'../results/{name_img}/'
     Parallel(n_jobs=-1)(delayed(detect_and_save_result_matches_csv)(train_img, query_img, var[0], var[1],detector, descriptor, params, path) for detector, params in zip(detectors, params_detectors) for descriptor in descriptors)

In [1199]:
# Parallel(n_jobs=-1)(delayed(comparing_keypoints_parallel)(df, df1_, df2_, idx1, kp1_col, kp2_col, cols1, cols2, extra_cols) for idx1 in df1_.index)

In [1185]:
params_sift_2 = {
    # 'nfeatures': 100,
    'nOctaveLayers' : 1,
    'contrastThreshold' : 0.08,
    'edgeThreshold' : 20,
    'sigma' : 1.6
}


In [1186]:
result = matchDetectedKeypoints(train_img_1_, query_img_1_, 3, 0.05, detector='sift', descriptor='brisk', dect_kargs=params_sift_2, des_kargs={})
final_img, qtd_kp_train, qtd_kp_query, ratio_test, var_dist, var_slope, qtd_matches, qtd_matches_dlowe, qtd_fp_dlowe = result 

print('---- Resultado ---- ')
print(f'Quantidade de keypoints train image: {qtd_kp_train}')
print(f'Quantidade de keypoints query image: {qtd_kp_query}')
print(f'Quantidade de matches: {qtd_matches}')
print(f'Valor ratio test D.Lowe: {ratio_test}')
print(f'Quantidade de matches filtro D.Lowe: {qtd_matches_dlowe}')
print(f'Quantidade de falso D.Lowe: {qtd_fp_dlowe}')
print(f'Variância da distância: {var_dist}')
print(f'Variância da inclinação da linha: {var_slope}')

# Show the final image 
cv2.imshow("Matches", final_img) 
cv2.waitKey()

---- Resultado ---- 
Quantidade de keypoints train image: 132
Quantidade de keypoints query image: 133
Quantidade de matches: 133
Valor ratio test D.Lowe: 0.66
Quantidade de matches filtro D.Lowe: 60
Quantidade de falso D.Lowe: 73
Variância da distância: 0.2038136775340516
Variância da inclinação da linha: 0.0004199397953293102


-1

# Comparando Fluxo Óptico

## Lucas-Kanade

In [1227]:
def calculate_distance_points(old_points, new_points):
    array_pts_train = np.asarray(old_points)
    array_pts_query = np.asarray(new_points)
    
    dists = [ np.linalg.norm(p1 - p2) for p1, p2 in zip(array_pts_train, array_pts_query)]

    return dists


def calculate_slope(pt1, pt2):
    slope = (pt2[1] - pt1[1]) / (pt2[0] - pt1[0]) if (pt2[0] - pt1[0]) != 0 else -1
    return slope


def calculate_slope_points(old_points, new_points):
    array_pts_train = np.asarray(old_points)
    array_pts_query = np.asarray(new_points)
    
    slopes = [ calculate_slope(p1, p2) for p1, p2 in zip(array_pts_train, array_pts_query)]

    return slopes


def filter_outliers(data, old_points, new_points):

    Q1 = np.percentile(data, 25) 
    Q2 = np.percentile(data, 50) 
    Q3 = np.percentile(data, 75)

    IQR = Q3 - Q1

    low_lim = Q1 - 1.5 * IQR
    up_lim = Q3 + 1.5 * IQR

    filtered_old = [ old_points[i] for i, d in enumerate(data) if d <= up_lim and d >= low_lim]
    filtered_new = [ new_points[i] for i, d in enumerate(data) if d <= up_lim and d >= low_lim]

    return filtered_old, filtered_new


def filter_by_variance(old_points, new_points, var_dist, var_slope):
    dists = calculate_distance_points(old_points, new_points)
    slopes = calculate_slope_points(old_points, new_points)

    filtered_old = [p for p in old_points]
    filtered_new = [p for p in new_points]

    var_dist_ = np.var(dists)
    var_slope_ = np.var(slopes)
    loop_limit = 100
    
    while(var_dist_ > var_dist or var_slope_ > var_slope):
        dists = calculate_distance_points(filtered_old, filtered_new)
        var_dist_ = np.var(dists)

        if var_dist_ > var_dist:
            filtered_old, filtered_new = filter_outliers(dists, filtered_old, filtered_new)
        
        slopes = calculate_slope_points(filtered_old, filtered_new)
        var_slope_ = np.var(slopes)

        if var_slope_ > var_slope:
            filtered_old, filtered_new = filter_outliers(slopes, filtered_old, filtered_new)
        
        if loop_limit > 0:
            loop_limit -= 1
        else:
            break

    return np.asarray(filtered_old), np.asarray(filtered_new)


def lucas_kanade_opflow(train_img, query_img, feature_params, lk_params, var_dist, var_slope):
    
    # Take first frame and find corners in it 
    old_frame = train_img.copy() 
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params) 

    # Create a mask image for drawing purposes 

    mask = np.zeros_like(old_frame, 'uint8') 
    
    frame = query_img.copy()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 

    # calculate optical flow 
    p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params) 

    # Select good points 
    good_new = p1[st == 1] 
    good_old = p0[st == 1]

    good_old_, good_new_ = filter_by_variance(good_old, good_new, var_dist, var_slope)
    dists = calculate_distance_points(good_old_, good_new_)
    slopes = calculate_slope_points(good_old_, good_new_)
    
    var_dist_ = np.var(dists)
    var_slope_ = np.var(slopes)
    
    qtd_matches = len(good_new)
    qtd_matches_filtered = len(good_new_)
    qtd_fp = qtd_matches - qtd_matches_filtered

    line = (0,255,0)
    point_true = (255,0,0)
    point_false = (0,0,255)

    # draw the tracks
    for i, (new, old) in enumerate(zip(good_new,  
                                        good_old)):
        a, b = new.ravel() 
        c, d = old.ravel()
        
        a, b = int(a), int(b)
        c, d = int(c), int(d)

        if new in good_new_:
            mask = cv2.line(mask, (a, b), (c, d), line, 2)     
            frame = cv2.circle(frame, (a, b), 5, point_true, -1) 
        else:
            mask = cv2.line(mask, (a, b), (c, d), line, 2)     
            frame = cv2.circle(frame, (a, b), 5, point_false, -1)
            
    img = cv2.add(frame, mask) 

    return img, var_dist_, var_slope_, qtd_matches, qtd_matches_filtered, qtd_fp
    


In [1235]:
def lk_and_save_result_matches_csv(train_img, query_img, features_params, lk_params, name_img):
    cols_features = [param for param in list(features_params.keys())]
    cols_lk = [param for param in list(lk_params.keys())]

    cols_name = cols_features + cols_lk

    cols_result = ['var_dist', 'var_slope', 'qtd_matches', 'qtd_matches_filtered', 'qtd_fp']

    df = pd.DataFrame(columns=cols_name+cols_result)
    
    keys_feat = list(features_params)
    keys_lk = list(lk_params)
    path = f'../results/{name_img}/lk_matches/'
    i = 0
    for values_feat in itertools.product(*map(features_params.get, keys_feat)):
        for values_lk in itertools.product(*map(lk_params.get, keys_lk)):
            try:
                result = lucas_kanade_opflow(train_img, query_img, dict(zip(keys_feat, values_feat)), 
                                            dict(zip(keys_lk, values_lk)), 0, 0) 
            except:
                result = [-1] + [-1 for i in range(len(cols_result))]

            df.loc[len(df)] = list(values_feat) + list(values_lk) + list(result[1:])
            
            path_img = path + str(i) + '.jpg'
            cv2.imwrite(path_img, result[0])
            i+=1
    
    path_csv = path + 'lk_matches.csv'
    df.to_csv(path_csv)

In [1236]:
import numpy as np 
import cv2 

query_img_1 = cv2.imread('../data/imgs_teste/IMG_20231217_075107.jpg') 
train_img_1 = cv2.imread('../data/imgs_teste/IMG_20231217_075101.jpg')

query_img_2 = cv2.imread('../data/imgs/dsc07632.jpg') 
train_img_2 = cv2.imread('../data/imgs/dsc07631.jpg')

query_img_3 = cv2.imread('../data/imgs/dsc02596.jpg') 
train_img_3 = cv2.imread('../data/imgs/dsc02595.jpg')

query_img_1_ = cv2.resize(query_img_1, (480,640))
train_img_1_ = cv2.resize(train_img_1, (480,640))

query_img_2_ = cv2.resize(query_img_2, (480,640))
train_img_2_ = cv2.resize(train_img_2, (480,640))

query_img_3_ = cv2.resize(query_img_3, (480,640))
train_img_3_ = cv2.resize(train_img_3, (480,640))

In [1237]:
features_params = {
    'maxCorners' : [25, 50, 100], 
    'qualityLevel' : [0.02, 0.05, 0.1],
    'minDistance' : [3, 7, 9] , 
    'blockSize' : [3, 7, 9],
    'useHarrisDetector' : [False]
} 
  
# Parameters for lucas kanade optical flow 
lk_params = {
    'winSize' : [(8, 8), (16, 16), (24, 24)], 
    'maxLevel': [2, 4, 8], 
    'criteria': [(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 
                              10, 0.03)]
} 

In [1238]:
name_imgs = ['IMG_20231217_075101', 'dsc07631', 'dsc02595']
train_imgs = [train_img_1_, train_img_2_, train_img_3_]
query_imgs = [query_img_1_, query_img_2_, query_img_3_]

Parallel(n_jobs=-1)(delayed(lk_and_save_result_matches_csv)(train_img, query_img, features_params, lk_params, name_img) for name_img, train_img, query_img in zip(name_imgs, train_imgs, query_imgs))

[None, None, None]

In [ ]:
import numpy as np 
import cv2 

train_img = cv2.imread('../data/imgs_teste/IMG_20231217_075101.jpg') 
query_img = cv2.imread('../data/imgs_teste/IMG_20231217_075107.jpg')
# query_img = cv2.imread('../data/imgs/dsc07632.jpg') 
# train_img = cv2.imread('../data/imgs/dsc07631.jpg')
train_img_ = cv2.resize(train_img, (480,640))
query_img_ = cv2.resize(query_img, (480,640))

# params for corner detection 
feature_params = dict( maxCorners = 100, 
                       qualityLevel = 0.02, 
                       minDistance = 7, 
                       blockSize = 7,
                       useHarrisDetector = False) 
  
# Parameters for lucas kanade optical flow 
lk_params = dict( winSize = (16, 16), 
                  maxLevel = 2, 
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 
                              10, 0.03)) 
  

lk_result = lucas_kanade_opflow(train_img_, query_img_, feature_params, lk_params, 0, 0)

img, var_dist, var_slope, qtd_matches, qtd_matches_filtered, qtd_fp = lk_result

print('---- Resultado ---- ')
print(f'Quantidade de matches: {qtd_matches}')
print(f'Quantidade de matches filtro outliers: {qtd_matches_filtered}')
print(f'Quantidade de falso positivo: {qtd_fp}')
print(f'Variância da distância: {var_dist}')
print(f'Variância da inclinação da linha: {var_slope}')

cv2.imshow('frame', img)
cv2.waitKey()


---- Resultado ---- 
Quantidade de matches: 100
Quantidade de matches filtro outliers: 91
Quantidade de falso positivo: 9
Variância da distância: 0.241999089717865
Variância da inclinação da linha: 0.00028355419635772705


-1

## Horn-Schunk 

In [1249]:
def draw_flow(img, flow, new_points_, step=24):
    h, w = img.shape[:2]
    y, x = np.mgrid[step / 2:h:step, step / 2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T
    lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)
    vis = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)

    line = (0,255,0)
    point_true = (255,0,0)
    point_false = (0,0,255)
    # cv2.polylines(vis, lines, 0, (0, 255, 0))
    
    for (x1, y1), (x2, y2) in lines:
        if np.asarray([x2, y2]) in new_points_:
            cv2.circle(vis, (x1, y1), 2, point_true, -1)
            cv2.line(vis, (x1, y1), (x2, y2), line, 1)
        else:
            cv2.circle(vis, (x1, y1), 2, point_false, -1)
            cv2.line(vis, (x1, y1), (x2, y2), line, 1)
    return vis


def return_points_hs(img, flow, step=24):
    h, w = img.shape[:2]
    y, x = np.mgrid[step / 2:h:step, step / 2:w:step].reshape(2, -1).astype(int)
    fx, fy = flow[y, x].T
    lines = np.vstack([x, y, x + fx, y + fy]).T.reshape(-1, 2, 2)
    lines = np.int32(lines + 0.5)

    old_points = np.asarray([[x1, y1] for (x1, y1), (x2, y2) in lines if calculate_distance_points([[x1, y1]], [[x2, y2]])[0] > 0.5])
    new_points = np.asarray([[x2, y2] for (x1, y1), (x2, y2) in lines if calculate_distance_points([[x1, y1]], [[x2, y2]])[0] > 0.5])

    return old_points, new_points


def horn_schunck_opflow(train_img, query_img, params_horn_schunck):
    # Convert to gray scale
    prvs = cv2.cvtColor(train_img, cv2.COLOR_BGR2GRAY)
    
    # Capture another frame and convert to gray scale
    next = cv2.cvtColor(query_img, cv2.COLOR_BGR2GRAY)

    # Optical flow is now calculated
    flow = cv2.calcOpticalFlowFarneback(prev=prvs, next=next,flow=None, flags=0, **params_horn_schunck)

    # Calculated fp points
    old_points, new_points = return_points_hs(next, flow, step=24)
    old_points_, new_points_ = filter_by_variance(old_points, new_points, 0.0, 0.0)
    dists = calculate_distance_points(old_points_, new_points_)
    slopes = calculate_slope_points(old_points_, new_points_)

    var_dist_ = np.var(dists)
    var_slope_ = np.var(slopes)

    qtd_matches = len(new_points)
    qtd_matches_filtered = len(new_points_)
    qtd_fp = qtd_matches - qtd_matches_filtered

    mapped_img = draw_flow(next, flow, new_points_, step=24)

    return mapped_img, var_dist_, var_slope_, qtd_matches, qtd_matches_filtered, qtd_fp
    

In [1250]:
def hs_and_save_result_matches_csv(train_img, query_img, hs_params, name_img):
    cols_name = [param for param in list(hs_params.keys())]

    cols_result = ['var_dist', 'var_slope', 'qtd_matches', 'qtd_matches_filtered', 'qtd_fp']

    df = pd.DataFrame(columns=cols_name+cols_result)
    
    keys = list(hs_params)
    path = f'../results/{name_img}/hs_matches/'
    
    for i, values in enumerate(itertools.product(*map(hs_params.get, keys))):
        try:
            result = horn_schunck_opflow(train_img, query_img, dict(zip(keys, values)))
        except:
            result = [-1] + [-1 for i in range(len(cols_result))]

        df.loc[len(df)] = list(values) + list(result[1:])
        
        path_img = path + str(i) + '.jpg'
        cv2.imwrite(path_img, result[0])
    
    path_csv = path + 'hs_matches.csv'
    df.to_csv(path_csv)

In [1269]:
import numpy as np 
import cv2 

query_img_1 = cv2.imread('../data/imgs_teste/IMG_20231217_075107.jpg') 
train_img_1 = cv2.imread('../data/imgs_teste/IMG_20231217_075101.jpg')

query_img_2 = cv2.imread('../data/imgs/dsc07632.jpg') 
train_img_2 = cv2.imread('../data/imgs/dsc07631.jpg')

query_img_3 = cv2.imread('../data/imgs/dsc02596.jpg') 
train_img_3 = cv2.imread('../data/imgs/dsc02595.jpg')

query_img_1_ = cv2.resize(query_img_1, (480,640))
train_img_1_ = cv2.resize(train_img_1, (480,640))

query_img_2_ = cv2.resize(query_img_2, (480,640))
train_img_2_ = cv2.resize(train_img_2, (480,640))

query_img_3_ = cv2.resize(query_img_3, (480,640))
train_img_3_ = cv2.resize(train_img_3, (480,640))

In [1270]:
params_horn_schunck = {
    'pyr_scale': [0.1, 0.5, 0.8],
    'levels': [3, 6, 9], 
    'winsize': [15, 30, 45],
    'iterations': [3, 15, 30], 
    'poly_n': [3, 6, 9], 
    'poly_sigma': [1.2, 1.6, 2.4]
    }

In [1271]:
name_imgs = ['IMG_20231217_075101', 'dsc07631', 'dsc02595']
train_imgs = [train_img_1_, train_img_2_, train_img_3_]
query_imgs = [query_img_1_, query_img_2_, query_img_3_]

Parallel(n_jobs=-1)(delayed(hs_and_save_result_matches_csv)(train_img, query_img, params_horn_schunck, name_img) for name_img, train_img, query_img in zip(name_imgs, train_imgs, query_imgs))

[None, None, None]

In [1268]:
# Importing libraries
import cv2
import numpy as np

# train_img = cv2.imread('../data/imgs_teste/IMG_20231217_075101.jpg') 
# query_img = cv2.imread('../data/imgs_teste/IMG_20231217_075107.jpg')
query_img = cv2.imread('../data/imgs/dsc07632.jpg') 
train_img = cv2.imread('../data/imgs/dsc07631.jpg')  
train_img_ = cv2.resize(train_img, (480,640))
query_img_ = cv2.resize(query_img, (480,640))

params_horn_schunck = dict(pyr_scale=0.5, levels=3, winsize=15,
                           iterations=3, poly_n=5, poly_sigma=1.2)

hs_optical = horn_schunck_opflow(train_img_, query_img_, params_horn_schunck)
mapped_img, var_dist, var_slope, qtd_matches, qtd_matches_filtered, qtd_fp = hs_optical

print('---- Resultado ---- ')
print(f'Quantidade de matches: {qtd_matches}')
print(f'Quantidade de matches filtro outliers: {qtd_matches_filtered}')
print(f'Quantidade de falso positivo: {qtd_fp}')
print(f'Variância da distância: {var_dist}')
print(f'Variância da inclinação da linha: {var_slope}')

cv2.imshow('frame2', mapped_img)
cv2.waitKey()


---- Resultado ---- 
Quantidade de matches: 475
Quantidade de matches filtro outliers: 309
Quantidade de falso positivo: 166
Variância da distância: 3.2450807925440697
Variância da inclinação da linha: 0.018452262684142417


-1